# Exploring the SFMTA Static GTFS

In [1]:
!cat SFMTA_Transit_Data_License_Agreement.txt

## Loading in all the files

In [40]:
import numpy as np
import pandas as pd

In [2]:
agency = pd.read_csv('agency.txt', sep=",", header=0)
calendar = pd.read_csv('calendar.txt', sep=",", header=0)
calendar_dates = pd.read_csv('calendar_dates.txt', sep=",", header=0)
fare_attributes = pd.read_csv('fare_attributes.txt', sep=",", header=0)
fare_rules = pd.read_csv('fare_rules.txt', sep=",", header=0)
routes = pd.read_csv('routes.txt', sep=",", header=0)
shapes = pd.read_csv('shapes.txt', sep=",", header=0)
stop_times = pd.read_csv('stop_times.txt', sep=",", header=0)
stops = pd.read_csv('stops.txt', sep=",", header=0)
trips = pd.read_csv('trips.txt', sep=",", header=0)

## Exploratory Data Analysis
Performing an exploration of each file, in alphabetical order, with the help of the GTFS reference

### agency [required]
Used to refer to agency information if more than one GTFS source is merged together.

In [3]:
agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
0,SFMTA,San Francisco Municipal Transportation Agency,http://www.sfmta.com,America/Los_Angeles,en


### calendar [conditionally required]
SFMTA does provide a calendar.txt which is only conditionally required.
* the dates for which this data applies is 9th February 2019 ~ 8th March 2019
* it appears that the provision of service on weekdays is a binary value - service is either provided on all weekdays or not provided on any weekday
* service_id is defined as a primary key under GTFS guidelines, it is also a foreign key for trips and calendar_dates

In [4]:
calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,1,1,1,1,1,1,0,0,20190209,20190308
1,2,0,0,0,0,0,1,0,20190209,20190308
2,3,0,0,0,0,0,0,1,20190209,20190308


### calendar_dates [conditionally required]
service_id here is a foreign key for calendar [conditionally required].

calendar_dates is specifying here that:
* on 21st January 2019, the saturday service has been removed
* on 21st January 2019, the sunday service has been added
* on 18th February 2019, the saturday service has been removed
* on 18th February 2019, the sunday service has been added


**My confusion:**
* **date is supposed to be the "Date when service exception occurs." but the 21st January 2019 which is Martin Luther King Jr. Day falls on a Monday**
* **date is supposed to be the "Date when service exception occurs." but the 18th February 2019 which is President's Day falls on a Monday**

In [5]:
calendar_dates

,service_id,date,exception_type
0,1,20190121,2
1,2,20190121,1
2,1,20190218,2
3,2,20190218,1


### fare_attributes [optional]
fare_id is a primary key here and a foreign key for fare_rules [optional]

payment_method:
* 0 - Fare is paid on board
* 1 - Fare must be paid before boarding

transfers:
* 0 - No transfers permitted on this fare.
* 1 - Riders may transfer once.
* 2 - Riders may transfer twice.
* empty - Unlimited transfers are permitted.

transfer_duration is the "length of time in seconds before a transfer expires":
* 5400 seconds = 90 minutes

In [6]:
fare_attributes

,fare_id,price,currency_type,payment_method,transfers,transfer_duration
0,1,2.75,USD,0,NaN,5400.0
1,2,7.00,USD,0,0.0,NaN


### fare_rules [optional]
fare_id is a foreign key for fare_attributes [optional].

route_id is a foreign key for routes [required].

fare_id and route_id are the only columns that feature non-null values.

origin_id, destination_id, and contains_id appear to be null objects but are categorised as floats.

They have been turned into null objects.

In [7]:
fare_rules.head()

,fare_id,route_id,origin_id,destination_id,contains_id
0,1,13392,NaN,NaN,NaN
1,1,13711,NaN,NaN,NaN
2,1,13712,NaN,NaN,NaN
3,1,13406,NaN,NaN,NaN
4,1,13407,NaN,NaN,NaN


In [56]:
print(fare_rules["origin_id"].unique())
print(fare_rules["destination_id"].unique())
print(fare_rules["contains_id"].unique())

[ nan]
[ nan]
[ nan]
<class 'numpy.float64'>


In [57]:
print(type(fare_rules["contains_id"][0]))

<class 'numpy.float64'>


In [58]:
fare_rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 5 columns):
fare_id           83 non-null int64
route_id          83 non-null int64
origin_id         0 non-null float64
destination_id    0 non-null float64
contains_id       0 non-null float64
dtypes: float64(3), int64(2)
memory usage: 3.3 KB


In [59]:
fare_rules["origin_id"] = np.nan
fare_rules["destination_id"] = np.nan
fare_rules["contains_id"] = np.nan

In [60]:
fare_rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 5 columns):
fare_id           83 non-null int64
route_id          83 non-null int64
origin_id         0 non-null float64
destination_id    0 non-null float64
contains_id       0 non-null float64
dtypes: float64(3), int64(2)
memory usage: 3.3 KB


### routes [required]

agency_id is a foreign key for agency.

route_short_name is often what "riders use to identify a route."

There are no route_descriptions or route_url.

route_color and route_text_color were left empty so they default to white and black respectively.

There are 3 out of 7 potential transportation types in route_type:
* 0 - Tram, Streetcar, Light rail. Any light rail or street level system within a metropolitan area.
* 3 - Bus. Used for short- and long-distance bus routes.
* 5 - Cable car. Used for street-level cable cars where the cable runs beneath the car.

In [11]:
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,13392,SFMTA,1,CALIFORNIA,NaN,3,,,
1,13711,SFMTA,10,TOWNSEND,NaN,3,,,
2,13712,SFMTA,12,FOLSOM-PACIFIC,NaN,3,,,
3,13406,SFMTA,14,MISSION,NaN,3,,,
4,13407,SFMTA,14R,MISSION RAPID,NaN,3,,,


In [12]:
routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 9 columns):
route_id            83 non-null int64
agency_id           83 non-null object
route_short_name    83 non-null object
route_long_name     83 non-null object
route_desc          0 non-null float64
route_type          83 non-null int64
route_url           83 non-null object
route_color         83 non-null object
route_text_color    81 non-null object
dtypes: float64(1), int64(2), object(6)
memory usage: 5.9+ KB


In [13]:
routes["route_type"].unique()

array([3, 5, 0])

In [61]:
print(routes["route_url"].unique())
print(routes["route_color"].unique())
print(routes["route_text_color"].unique())

[' ']
[' ']
[' ' nan]


In [62]:
routes["route_text_color"] = "000000"
routes["route_color"] = "FFFFFF"

In [63]:
print(routes["route_color"].unique())
print(routes["route_text_color"].unique())

['FFFFFF']
['000000']


### shapes [optional]
There are no null objects in shapes.

The shape_id identifies the shape and the shape_pt_sequence does not have to increase chronologically.

There are 1953 unique shape_id entries.

In [17]:
shapes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233282 entries, 0 to 233281
Data columns (total 5 columns):
shape_id               233282 non-null int64
shape_pt_lon           233282 non-null float64
shape_pt_lat           233282 non-null float64
shape_pt_sequence      233282 non-null int64
shape_dist_traveled    233282 non-null int64
dtypes: float64(2), int64(3)
memory usage: 8.9 MB


In [37]:
shapes.head()

,shape_id,shape_pt_lon,shape_pt_lat,shape_pt_sequence,shape_dist_traveled
0,159273,-122.396968,37.795436,1,0
1,159273,-122.396784,37.795471,2,17
2,159273,-122.396598,37.794593,3,116
3,159273,-122.397809,37.794439,4,224
4,159273,-122.398983,37.794289,5,329


In [33]:
len(shapes["shape_id"].unique())

1953

### stop_times [required]
trip_id is a foreign key for trips [required].

stop_headsign and shape_dist_traveled contain only null objects.

for pickup_type and drop_off_type, "0 or empty - Regularly scheduled drop off."

instead of empty strings, they have been filled with 0.

In [35]:
stop_times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109813 entries, 0 to 1109812
Data columns (total 9 columns):
trip_id                1109813 non-null int64
arrival_time           1109813 non-null object
departure_time         1109813 non-null object
stop_id                1109813 non-null int64
stop_sequence          1109813 non-null int64
stop_headsign          1109813 non-null object
pickup_type            1109813 non-null object
drop_off_type          1109813 non-null object
shape_dist_traveled    1109813 non-null object
dtypes: int64(3), object(6)
memory usage: 76.2+ MB


In [36]:
stop_times.head(10)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,8204629,17:28:00,17:28:00,4015,1,,,,
1,8204629,17:29:06,17:29:06,6294,2,,,,
2,8204629,17:30:22,17:30:22,6290,3,,,,
3,8204629,17:31:00,17:31:00,6314,4,,,,
4,8204629,17:32:23,17:32:23,6307,5,,,,
5,8204629,17:34:02,17:34:02,6302,6,,,,
6,8204629,17:35:22,17:35:22,6299,7,,,,
7,8204629,17:36:33,17:36:33,6316,8,,,,
8,8204629,17:38:00,17:38:00,6312,9,,,,
9,8204629,17:39:22,17:39:22,6315,10,,,,


In [39]:
print(stop_times["stop_headsign"].unique())
print(stop_times["pickup_type"].unique())
print(stop_times["drop_off_type"].unique())
print(stop_times["shape_dist_traveled"].unique())

[' ']
[' ']
[' ']
[' ']


In [64]:
stop_times["pickup_type"] = 0
stop_times["drop_off_type"] = 0

In [65]:
stop_times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109813 entries, 0 to 1109812
Data columns (total 9 columns):
trip_id                1109813 non-null int64
arrival_time           1109813 non-null object
departure_time         1109813 non-null object
stop_id                1109813 non-null int64
stop_sequence          1109813 non-null int64
stop_headsign          1109813 non-null object
pickup_type            1109813 non-null int64
drop_off_type          1109813 non-null int64
shape_dist_traveled    1109813 non-null object
dtypes: int64(5), object(4)
memory usage: 76.2+ MB


### stops [required]
no stop_desc or stop_url was provided, filled in with null values.

zone_id "identifies the fare zone for a stop. This field is required if providing fare information using fare_rules.txt, otherwise it is optional." It would be a foreign key for origin_id, destination_id, or contains_id in fare_rules but fare_rules uses route_id to determine the fare. zone_id has also been filled in with null values.

In [45]:
stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 8 columns):
stop_id      3491 non-null int64
stop_code    3491 non-null int64
stop_name    3491 non-null object
stop_desc    3491 non-null object
stop_lat     3491 non-null float64
stop_lon     3491 non-null float64
zone_id      3491 non-null object
stop_url     3491 non-null object
dtypes: float64(2), int64(2), object(4)
memory usage: 218.3+ KB


In [46]:
stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url
0,390,10390,19th Avenue & Holloway St,,37.721190,-122.475096,,
1,913,10913,DUBLIN ST & LAGRANDE AVE,,37.719192,-122.425802,,
2,3003,13003,2nd St & Brannan St,,37.781827,-122.391945,,
3,3004,13004,2nd St & Brannan St,,37.781854,-122.392232,,
4,3009,13009,2nd St & Harrison St,,37.784532,-122.395325,,


In [67]:
print(stops["stop_desc"].unique())
print(stops["zone_id"].unique())
print(stops["stop_url"].unique())

[' ']
[' ']
[' ']


In [70]:
stops["zone_id"] = np.nan

In [71]:
stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 8 columns):
stop_id      3491 non-null int64
stop_code    3491 non-null int64
stop_name    3491 non-null object
stop_desc    3491 non-null object
stop_lat     3491 non-null float64
stop_lon     3491 non-null float64
zone_id      0 non-null float64
stop_url     3491 non-null object
dtypes: float64(3), int64(2), object(3)
memory usage: 218.3+ KB


### trips
route_id is a foreign key for routes [required].

service_id is a foreign key for calendar or calendar_dates.

trip_id is a primary key.

direction_id is binary where 0 and 1 represents one direction of travel and the other the opposite.

block_id --> "In public transport, blocking is the practice of dividing the parts of a scheduled route among vehicles and drivers. It follows the process of dividing the route into trips. In blocking, these trips are pieced together into blocks that are relatively contiguous in space and time."

In [76]:
trips.head(10)

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,13392,1,8318856,Geary + 33rd Avenue,0,122,159274
1,13392,1,8318855,Geary + 33rd Avenue,0,112,159274
2,13392,1,8318854,Geary + 33rd Avenue,0,121,159274
3,13392,1,8318853,Geary + 33rd Avenue,0,111,159274
4,13392,1,8318852,Geary + 33rd Avenue,0,120,159274
5,13392,1,8318935,Geary + 33rd Avenue,0,151,159275
6,13392,1,8318784,Geary + 33rd Avenue,0,115,159275
7,13392,1,8318783,Geary + 33rd Avenue,0,114,159275
8,13392,1,8318782,Geary + 33rd Avenue,0,112,159275
9,13392,1,8318781,Geary + 33rd Avenue,0,113,159275


In [73]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29290 entries, 0 to 29289
Data columns (total 7 columns):
route_id         29290 non-null int64
service_id       29290 non-null int64
trip_id          29290 non-null int64
trip_headsign    29290 non-null object
direction_id     29290 non-null int64
block_id         29290 non-null int64
shape_id         29290 non-null int64
dtypes: int64(6), object(1)
memory usage: 1.6+ MB


In [74]:
len(trips["trip_id"].unique())

29290